# Setup

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import json
import numpy as np
import cv2
from google.cloud import storage
import os
import time
from uuid import uuid4
import pickle
import datetime


# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device: %s" % device)
# device = 'cpu'

device: cuda


In [2]:
with open('annotations/valid.json') as f:
    test_data = json.load(f)
with open('annotations/train.json') as f:
    train_data = json.load(f)
    
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "pose-estimation-2-dc39bc540ba3.json"
    
storage_client = storage.Client("pose_estimation_2")
bucket = storage_client.get_bucket('pose_estimation_2_dataset_mpii')

NUM_TRAIN = 256
NUM_TEST = 2958

# Hyperparameters

In [3]:
batch_size = 16
epochs = 30
learning_rate = 0.0001
optimizer_type = 'ADAM'

hyperparam_string = f'batch_size: {batch_size}, epochs: {epochs}, lr: {learning_rate}, optimizer: {optimizer_type}'

# Train Loop

In [4]:
import modules
import gc

from modules.unipose import UniPose
from modules.criterion.distribution_difference_loss import DistributionDifferenceLoss 
from modules.criterion.joint_max_mse_loss import JointMaxMSELoss
from gaussians import Gaussians

model = UniPose().to(device)
criterion = DistributionDifferenceLoss(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate) if optimizer_type == 'ADAM' else None
gaussian = Gaussians()

In [5]:
load_start = time.time()

kpt_list     = []

torch_image = torch.zeros(NUM_TRAIN, 368, 368, 3, dtype=torch.half)
kpt_tensor = torch.zeros(torch_image.shape, dtype=torch.half)
torch_image.requires_grad = False
kpt_tensor.requires_grad = False

# For each image, load the image
for i in range(NUM_TRAIN):
    img_name = train_data[i]['image']

    blob = bucket.blob('MPII/images/' +  img_name)
    blob.content_type = 'image/jpeg'
    image = np.asarray(bytearray(blob.download_as_string()))
    img = cv2.imdecode(image, cv2.IMREAD_UNCHANGED)

    kpt = np.asarray(train_data[i]['joints'], dtype=np.int32)

    if img.shape[0] != 368 or img.shape[1] != 368:
        kpt[:,0] = kpt[:,0] * (368/img.shape[1])
        kpt[:,1] = kpt[:,1] * (368/img.shape[0])
        img = cv2.resize(img,(368,368))
        img = np.array(img)

    kpt_list.append(kpt)
    torch_image[i,:,:,:] = torch.HalfTensor(img)
    
    if i % 10 == 0:
        print(f'Loaded {i+1} images')

image_load_time = time.time()

# construct image tensor and label tensor
# torch_image = torch.Tensor(imagelist)
torch_image = torch_image.permute(0, 3, 1, 2)
expected_maps = gaussian.expected_to_gaussian(kpt_list)
torch_image.requires_grad = False
expected_maps.requires_grad = False

tensor_build_time = time.time()

print(f'Image Load:{image_load_time - load_start}, Tensor Build:{tensor_build_time - image_load_time}')
print(f'Loaded {NUM_TRAIN} images, shape is {torch_image.shape}, kpt shape is {expected_maps.shape}')

Loaded 1 images
Loaded 11 images
Loaded 21 images
Loaded 31 images
Loaded 41 images
Loaded 51 images
Loaded 61 images
Loaded 71 images
Loaded 81 images
Loaded 91 images
Loaded 101 images
Loaded 111 images
Loaded 121 images
Loaded 131 images
Loaded 141 images
Loaded 151 images
Loaded 161 images
Loaded 171 images
Loaded 181 images
Loaded 191 images
Loaded 201 images
Loaded 211 images
Loaded 221 images
Loaded 231 images
Loaded 241 images
Loaded 251 images
Image Load:33.69359350204468, Tensor Build:42.36976504325867
Loaded 256 images, shape is torch.Size([256, 3, 368, 368]), kpt shape is torch.Size([256, 16, 368, 368])


In [7]:
from sklearn.model_selection import train_test_split

train_torch_image, val_torch_image = train_test_split(torch_image, test_size = 0.25)
train_expected_maps, val_expected_maps = train_test_split(expected_maps, test_size = 0.25)

In [8]:
def train(epoch, model, criterion, optimizer, train_torch_image, train_expected_maps, batch_size):
    epoch_loss = []
    # For each batch
    for start_i in range(0, len(train_torch_image), batch_size):
        
        start_time = time.time()

#         torch_image_batch = torch_image[ start_i: start_i + batch_size ,:,:,:].to(device)
#         map_batch         = expected_maps[ start_i: start_i + batch_size ,:,:]
        
        # Loading in the training images from train/test splits
        torch_image_batch = train_torch_image[ start_i: start_i + batch_size ,:,:,:].to(torch.float32).to(device)
        map_batch         = train_expected_maps[ start_i: start_i + batch_size ,:,:].to(torch.float32).to(device)

        tensor_transfer_time = time.time()
        
        # Train on batch
        
        optimizer.zero_grad()

        out = model(torch_image_batch)

        forward_pass_time = time.time()
        
        batch_loss = criterion(out, map_batch)
        
        loss_function_time = time.time()
        
        epoch_loss.append(batch_loss.item())
        
        batch_loss.backward()
        
        backprop_time = time.time()

        optimizer.step()
        
        optimizer_time = time.time()

#         print(f'Epoch: {epoch}, Batch: {start_i // batch_size}, Batch Distribution Difference Loss: {batch_loss}, JointMaxMSELoss (to see if model is working): {alt_criterion(out, kpt_batch.to(device))}')
        print(f'Epoch: {epoch}, Batch: {start_i // batch_size}, Training Batch Distribution Difference Loss: {batch_loss}')
        print(f'Tensor Transfer:{tensor_transfer_time - start_time}, Forward:{forward_pass_time - tensor_transfer_time}, Criterion:{loss_function_time - forward_pass_time}')
        print(f'Backward:{backprop_time - loss_function_time}, Optimizer_Step:{optimizer_time - backprop_time}, Total:{optimizer_time - start_time}')
        
        gc.collect()
        torch.cuda.empty_cache()
    return epoch_loss

In [9]:
def validation(epoch, model, criterion, val_torch_image, val_expected_maps, batch_size):
    epoch_loss = []
    # For each batch
    for start_i in range(0, len(val_torch_image), batch_size):
        
        start_time = time.time()

#         torch_image_batch = torch_image[ start_i: start_i + batch_size ,:,:,:].to(device)
#         map_batch         = expected_maps[ start_i: start_i + batch_size ,:,:]
        
        # Loading in the training images from train/test splits
        torch_image_batch = val_torch_image[ start_i: start_i + batch_size ,:,:,:].to(device)
        map_batch         = val_expected_maps[ start_i: start_i + batch_size ,:,:].to(device)

        tensor_transfer_time = time.time()
        
        # Train on batch
        with torch.no_grad():
            out = model(torch_image_batch)

            forward_pass_time = time.time()

            batch_loss = criterion(out, map_batch)

            loss_function_time = time.time()
            
        epoch_loss.append(batch_loss.item())

            
#         print(f'Epoch: {epoch}, Batch: {start_i // batch_size}, Batch Distribution Difference Loss: {batch_loss}, JointMaxMSELoss (to see if model is working): {alt_criterion(out, kpt_batch.to(device))}')
        print(f'Epoch: {epoch}, Batch: {start_i // batch_size}, Validation Batch Distribution Difference Loss: {batch_loss}')
        print(f'Tensor Transfer:{tensor_transfer_time - start_time}, Forward:{forward_pass_time - tensor_transfer_time}, Criterion:{loss_function_time - forward_pass_time}')

        gc.collect()
        torch.cuda.empty_cache()
    return epoch_loss


In [10]:
train_epoch_losses = []
val_epoch_losses = []

gc.collect()
torch.cuda.empty_cache()

uuid_string = str(datetime.datetime.now())

# For each epoch
for epoch in range(epochs):

    train_epoch_loss = train(epoch, model, criterion, optimizer, train_torch_image, train_expected_maps, batch_size)
    val_epoch_loss = validation(epoch, model, criterion, val_torch_image, val_expected_maps, batch_size)
    
    print(f'Epoch: {epoch}, Training Average Batch Loss: {sum(train_epoch_loss) / len(train_epoch_loss)}')
    print(f'Epoch: {epoch}, Validation Average Batch Loss: {sum(val_epoch_loss) / len(val_epoch_loss)}')
    
    train_epoch_losses.append(train_epoch_loss)
    val_epoch_losses.append(val_epoch_loss)
    
    with open(f'epoch{epoch}.txt', 'a') as f:
        print(hyperparam_string, file=f)
        for e in range(epoch + 1):
            print(f'Epoch: {e}, Training Average Batch Loss: {sum(train_epoch_losses[e]) / len(train_epoch_losses[e])}', file=f)
            print(f'Epoch: {e}, Validation Average Batch Loss: {sum(val_epoch_losses[e]) / len(val_epoch_losses[e])}', file=f)
        print(f'Epoch: {e}, Training Epoch Losses: {train_epoch_losses}', file=f)
        print(f'Epoch: {e}, Validation Epoch Loss: {val_epoch_losses}', file=f)
        
    f_name = f'epoch{epoch}.txt'
    blob = bucket.blob(f"training_output/{uuid_string}/epoch{epoch}.txt")
    blob.upload_from_filename(f_name)
    os.remove(f_name)
        
    f_name = f'epoch{epoch}.pkl'
    pickle.dump(model, open(f_name, 'wb'))
    
    blob = bucket.blob(f"training_output/{uuid_string}/epoch{epoch}.pkl")
    blob.upload_from_filename(f_name)
    os.remove(f_name)

RuntimeError: CUDA out of memory. Tried to allocate 68.00 MiB (GPU 0; 4.00 GiB total capacity; 2.45 GiB already allocated; 49.40 MiB free; 2.51 GiB reserved in total by PyTorch)

In [ ]:

f_name = 'finalized_model.pkl'
pickle.dump(model, open(f_name, 'wb'))

blob = bucket.blob(f"training_output/{uuid_string}/final.pkl")
blob.upload_from_filename(f_name)
os.remove(f_name)